# Hopfield Network With Hashing - Dayan & Abbott

The following code takes as input .wav sound files and transforms each of them into MFCC vectors. These vectors are then each transformed into a hash. They are then used to train a Hopfield network. In this way, each sound vector becomes a memory pattern that can be accessed even if slightly corrupted.

This is a memory mechanism in a form of a Hopfield network. The stored items are called memory patterns. They are retrieved by a process of the input that is presented to the network dynamics which at some time step reaches a fixed stable point. This means that the input item has been recognized (i.e. there is a memory pattern identical or very similar to it).

Even noisy sounds or those corrupted to some extent can be accessed. In other words, if the input is $x_1 + \delta$ and the stored item is $x_1$, the network will still reach the fixed point of $x_1$ if $\delta$ is small enough.

Additionally, for storage purposes, sounds are transformed each into a hash - with this we reduce their dimensionality. This means we increase the storage capacity. 

In [14]:
# First, we load some dependencies.
import numpy as np
import math
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import sys
import glob
import random

In [15]:
# Folder with some wav files to test this script.
folder_train = "./waveforms/"

First, we need to transform the sounds into a readable format that can be used for hashing.

In [16]:
# Go through the folder and find all (and only) files ending with .wav
# Here, we transform each .wav file into MFCCs and then flatten them into one vector
# We do this because we want one hash per .wav file
#
# Arguments: sound folder
# Returns: a list of flattened MFCC vectors

def make_mfcc(folder):
    vectors = []
    for file in glob.glob(folder + "*.wav", recursive=True):
        (rate,sig) = wav.read(file)
        mfcc_feat = mfcc(sig,rate)
        vect = mfcc_feat.flatten()
        vectors.append(vect)
    return vectors

We will transform each sound (that is, each sound transformed into MFCC vectors, then flattened into one vector) into a hash.

In [17]:
# Transform a vector of speech into a hash
# The hash will be a matrix of the dimension = k*m
# We choose a random number k of units of the vector.
# And look for the highest value and turn it into 1.
# Everything else is 0.
# We thus get sparse matrices.
# We do this m times. Final output is h=k*m.


def get_hash(vector, k, m):
    d = len(vector)
    p = np.zeros((m,k,))
    for i in range(m):
        p[i] = np.random.permutation(d)[:k]
        
    h = np.zeros((m,k,))
    for i in range(m):
        ix = np.argmax(p[i])
        hi = np.zeros(k)
        hi[ix] = 1
        h[i] = hi
    h = np.hstack(h)
    return h

## Test:
# V = make_mfcc(folder)
# get_hash(V[1], 5, 3)

# Principle
# - Algo: inputs of dimension d, params k, m (hash dim=k*m)
#   - pre-processing: 
#       p=[]; 
#       for i=1:m: 
#           p[i] = random_perm(d)[:k]
#   - getting hash for X: 
#       h = []
#       for i=1:m:
#         ix = argmax(X[p[i]])
#         hi = zeros(k)
#         hi[ix] = 1
#         h = h + hi
#   -> i.e. there is a local WTA on m sets of 
#   randomly chosen k-tuple of dims -> hash is of length mk with exactly m ones.

Hopfield network consists of a symmetric recurrent weight matrix that is trained with memory patterns (presented as hash vectors) we want to store. The weight matrix is trained with those patterns such that each of them becomes a fixed point of the network. Once we want to "retrieve" a memory pattern, we need to find one of the fixed points.

In [18]:
# Function for the matrix M (symmetric recurrent weight matrix)
#
# Arguments: 
# lmbda (eigenvalue represented as a lambda), alpha (amount of active units),
# c (constant value of active components, inactive have 0), 
# N (number of neurons), V (list of vectors in a hash form)

def get_m(lmbda, alpha, c, N, V):
    
    # n is a vector of ones
    n = np.ones(N)

    vect_sum = np.zeros((N,N))
    for vect in V:
        outer_prod = np.outer((vect - alpha * c * n),(vect - alpha * c * n))
        vect_sum += outer_prod

    m = (lmbda / (pow(c,2)*alpha*N*(1-alpha))) \
        * vect_sum \
        - (np.outer(n,n) / (alpha*N))
    return m

We need to determine what is a fixed point of the network. This is done 

In [19]:
# Function for the fixed point
# Memory pattern satisfies v_m = F(M * v_m) (i.e. is a fixed point)
# We use a sigmoid function as F: F = 1/(1+np.exp(-x))

def convergence_criterion(x0, x1, tau):
#     return math.isclose(x0, x1, rel_tol=tau) 
    return np.allclose(x0, x1, rtol=tau)

def fixed_point(x0, m, tau, i):
    x1 = 1/(1 + np.exp(- np.inner(m,x0) ))
    while convergence_criterion(x0, x1, tau) == False:
        i += 1
        return fixed_point(x1, m, tau, i)
    print(i)
    return x1

#     if F == None:
#         x1 = 1/(1 + np.exp(- np.inner(m,x0) ))
#     else:
#         x1 = F*x0
    
#     if convergence_criterion(x0, x1, tau):
#         return x1
#     else:
#         i += 1
#         print(i)
#         return fixed_point(x1, tau, F)

In [20]:
# Test:

k = 5
m = 3
lmbda = 0.1
alpha = 0.6
c = 1
N = 15
V =[]

mfccs_vectors = make_mfcc(folder_train)
for vect in mfccs_vectors:
    v = get_hash(vect, 5, 3)
    V.append(v)

m = get_m(lmbda, alpha, c, N, V)


In [21]:
# Test to get to fixed poind

tau1 = 0.000001
tau2 = 0.1
# x0_1 is a vector similar to a memory pattern that is stored
x0_1 = np.array([0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.])
# x0_2 is a vector very different from any memory pattern stored
x0_2 = np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

print(fixed_point(x0_1, m, tau1, 0))

print(fixed_point(x0_1, m, tau2, 0))

# This is problematic: x0_2 is not even nearly similar to any of the stored patterns, 
# yet w reach the local minimum/fixed point

print(fixed_point(x0_2, m, tau1, 0))

print(fixed_point(x0_2, m, tau2, 0))

11
[0.39334329 0.40984807 0.37696892 0.3932346  0.4100676  0.40984807
 0.40973644 0.39355551 0.4100676  0.36077461 0.37783568 0.4100676
 0.40973201 0.42588487 0.36087596]
2
[0.39675003 0.41283273 0.38077336 0.39663041 0.41307817 0.41283273
 0.4127123  0.39697942 0.41307817 0.36494006 0.38169337 0.41307817
 0.41271117 0.42838444 0.36505202]
11
[0.39334336 0.40984814 0.37696901 0.39323467 0.41006766 0.40984814
 0.4097365  0.39355558 0.41006766 0.36077469 0.37783576 0.41006766
 0.40973207 0.42588493 0.36087605]
3
[0.39589927 0.41209422 0.37981404 0.39578245 0.41232934 0.41209422
 0.41197482 0.39612637 0.41232934 0.36388323 0.38073501 0.41232934
 0.41197149 0.42776497 0.3639927 ]


In [22]:
print(m)

[[-0.03222222 -0.07111111 -0.10444444 -0.08777778 -0.07111111 -0.07111111
  -0.04333333 -0.08777778 -0.07111111 -0.09333333 -0.07666667 -0.07111111
  -0.07111111 -0.05444444 -0.09333333]
 [-0.07111111 -0.02666667 -0.08777778 -0.07111111 -0.05444444 -0.02666667
  -0.05444444 -0.07111111 -0.05444444 -0.10444444 -0.06       -0.05444444
  -0.05444444 -0.03777778 -0.10444444]
 [-0.10444444 -0.08777778 -0.03777778 -0.10444444 -0.08777778 -0.08777778
  -0.08777778 -0.07666667 -0.08777778 -0.08222222 -0.12111111 -0.08777778
  -0.06       -0.07111111 -0.08222222]
 [-0.08777778 -0.07111111 -0.10444444 -0.03222222 -0.07111111 -0.07111111
  -0.07111111 -0.06       -0.07111111 -0.09333333 -0.07666667 -0.07111111
  -0.07111111 -0.05444444 -0.09333333]
 [-0.07111111 -0.05444444 -0.08777778 -0.07111111 -0.02666667 -0.05444444
  -0.05444444 -0.07111111 -0.02666667 -0.10444444 -0.08777778 -0.02666667
  -0.05444444 -0.03777778 -0.10444444]
 [-0.07111111 -0.02666667 -0.08777778 -0.07111111 -0.05444444 -0.